## 데이터 다운로드

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [2]:

import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

In [3]:
import numpy as np

In [4]:
!wget 'https://bit.ly/3dD5MU9'

import zipfile
with zipfile.ZipFile('3dD5MU9', 'r') as existing_zip:
    existing_zip.extractall('data')

--2021-06-03 14:44:31--  https://bit.ly/3dD5MU9
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z [following]
--2021-06-03 14:44:31--  https://drive.google.com/uc?export=download&id=1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z
Resolving drive.google.com (drive.google.com)... 74.125.132.113, 74.125.132.101, 74.125.132.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.132.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qu68qqt0usqesc5oora9n23fotj6ttui/1622731425000/00192245294648390361/*/1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z?e=download [following]
--2021-06-03 14:44:36--  https://doc-14-94-docs.googleusercontent.com/docs/securesc/

In [5]:
import pandas as pd
energy = pd.read_csv('data/energy.csv')
dangjin_fcst = pd.read_csv('./data/dangjin_fcst_data.csv')
ulsan_fcst = pd.read_csv('./data/ulsan_fcst_data.csv')
pd.read_csv('data/dangjin_fcst_data.csv')

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
...,...,...,...,...,...,...,...
162203,2021-03-01 08:00:00,52.0,7.0,40.0,3.2,187.0,1.0
162204,2021-03-01 08:00:00,55.0,8.0,40.0,4.5,217.0,1.0
162205,2021-03-01 08:00:00,58.0,5.0,55.0,2.2,210.0,1.0
162206,2021-03-01 08:00:00,61.0,1.0,80.0,1.9,164.0,1.0


## 데이터 병합

In [6]:
def to_date(x):
    return timedelta(hours=x)

In [7]:
def generate_df(df_):
    df = df_.copy()

    df['Forecast_time'] = pd.to_datetime(df['Forecast time'])

    # 20 시
    a = df[df["forecast"] == 4.0]
    a = a[a["Forecast time"].apply(lambda x: "20:00:00" in x)]
    a.loc[:, 'Forecast_time'] = a.loc[:, 'Forecast_time'] + a.loc[:, 'forecast'].map(to_date)

    # 23 시
    b = df[df["forecast"] <= 22]
    b = b[b["Forecast time"].apply(lambda x: "23:00:00" in x)]
    b.loc[:, 'Forecast_time'] = b.loc[:, 'Forecast_time'] + b.loc[:, 'forecast'].map(to_date)

    # 병합
    c = pd.concat([a, b])
    print(f"20시 사용 데이터 길이 : {len(a)}")
    print(f"23시 사용 데이터 길이 : {len(b)}")
    print(f"합친 데이터 길이 : {len(c)}")
    print()

    # 정렬
    c.sort_values(by=['Forecast_time'], inplace=True)
    c = c[['Forecast_time', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]

    return c

In [8]:
dangjin_filled = generate_df(dangjin_fcst)
ulsan_filled = generate_df(ulsan_fcst)

20시 사용 데이터 길이 : 1096
23시 사용 데이터 길이 : 7672
합친 데이터 길이 : 8768

20시 사용 데이터 길이 : 1096
23시 사용 데이터 길이 : 7672
합친 데이터 길이 : 8768



## 보간

In [9]:
def interpolate_df(df, method='linear'):
    new_df = pd.DataFrame()
    new_df['Forecast_time'] = pd.date_range(start=df['Forecast_time'].iloc[0], end=df['Forecast_time'].iloc[-1], freq='H')
    new_df = pd.merge(new_df, df, on='Forecast_time', how='outer')
    return new_df.interpolate(method=method)

In [10]:
dangjin_interpolated = interpolate_df(dangjin_filled, method='linear')
ulsan_interpolated = interpolate_df(ulsan_filled, method='linear')

In [11]:
def train_datast(energy_df, fcst_df, target):
    # 일기 예보 있는 날짜만 선택
    energy = energy_df.loc[24:]
    energy.index = range(energy.shape[0])
    
    # 발전량 데이터가 있는 날짜만 선택
    fcst = fcst_df
    fcst.index = range(fcst.shape[0])
    
    # 발전량과 일기예보 연결
    concat_df = pd.concat([energy, fcst], axis=1)
    
    
    # 예보 시간, 날짜, 기상 예보 및 발전량 선택
    feature_df = concat_df[['Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud', target]]
    feature_df.fillna(0,inplace = True)

    return np.array(feature_df[:-22])


In [12]:
dangjin_interpolated['Forecast_time'] = dangjin_interpolated['Forecast_time'].astype('str')
ulsan_interpolated['Forecast_time'] = ulsan_interpolated['Forecast_time'].astype('str')

In [13]:
df = train_datast(energy, dangjin_interpolated, target='dangjin_floating')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


## 하이퍼 파라미터 


In [14]:
# hyper parameters
seq_length = 24 # 입력 시퀀스 길이
data_dim = 6 # 입력 데이터 차원 (변수 갯수)
hidden_dim = 30 #(출력 데이터의 차원)
output_dim = 1 #(최종 예측 데이터 차원)
learning_rate = 0.01
iterations = 3000

In [15]:
train_size = int(len(df) * 1)
train_set = df[:train_size]
test_set = df[train_size-672:]
train_set.shape, test_set.shape

((26280, 6), (672, 6))

## MinMax 정규화


In [ ]:
'''scaler = MinMaxScaler()
scaler.fit(train_set)
train_set = scaler.transform(train_set)
test_set = scaler.transform(test_set)'''

'scaler = MinMaxScaler()\nscaler.fit(train_set)\ntrain_set = scaler.transform(train_set)\ntest_set = scaler.transform(test_set)'

In [16]:
def make_sequence(time_series, seq_length):
    dataX = []
    dataY = []

    for i in range(0, len(time_series) - seq_length) :
        _x = time_series[i:i + seq_length, : ] 
        _y = time_series[i+ seq_length, [-1]]
        

        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

In [17]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True, bidirectional=True)
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim*2, 20),
            torch.nn.Linear(20, 10),
            torch.nn.Linear(10, output_dim)
        )
    def forward(self, x):
        x, (hidden, cell) = self.rnn(x)
        x = self.layers(x[:, -1, ])
        return x
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net(data_dim, hidden_dim, output_dim, 4).to(device)

In [18]:
# train-test dataset to input

trainX, trainY = make_sequence(train_set, seq_length)
testX, testY = make_sequence(test_set, seq_length)

# convert to tensor
trainX_tensor = torch.FloatTensor(trainX).to(device)
trainY_tensor = torch.FloatTensor(trainY).to(device)

testX_tensor = torch.FloatTensor(testX).to(device)
testY_tensor = torch.FloatTensor(testY).to(device)

In [ ]:
!pip3 install adamp

In [ ]:
from adamp import AdamP

In [ ]:
'''from sklearn.model_selection import KFold

kf = KFold(n_splits=3, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(trainX):
    folds.append((train_idx, valid_idx))    

for fold in range(3):
    best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]

    # cuda cache 초기화
    torch.cuda.empty_cache()
    
    #train fold, validation fold 분할
    train_answer = trainY.iloc[train_idx]
    test_answer  = trainY.iloc[valid_idx]

    train_df = trainX.iloc[train_idx]
    test_df = trainX.iloc[valid_idx]
    break'''

AttributeError: ignored

In [ ]:
# loss & optimizer setting
criterion = torch.nn.MSELoss()
optimizer = AdamP(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
# start training
net.train()
best_loss = int(1e9)

for i in range(iterations):
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_model = net
    scheduler.step()

best_model.eval()
dangjin_floating_pred = []
for i in range(672):
    x_input  = df[-672-24+i:-672+i].reshape(1,24,6)

    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = best_model(x_input).cpu().detach().numpy()

    dangjin_floating_pred.append(predict[0][0])
    new_input = predict.reshape((1))
    df[-672-24+i:-672+i][-1][-1] = new_input[0]
  

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
df = train_datast(energy, dangjin_interpolated, target='dangjin_warehouse')

train_size = int(len(df) * 1)
train_set = df[:train_size]
test_set = df[train_size-672:]
train_set.shape, test_set.shape


# train-test dataset to input

trainX, trainY = make_sequence(train_set, seq_length)
testX, testY = make_sequence(test_set, seq_length)

# convert to tensor
trainX_tensor = torch.FloatTensor(trainX).to(device)
trainY_tensor = torch.FloatTensor(trainY).to(device)

testX_tensor = torch.FloatTensor(testX).to(device)
testY_tensor = torch.FloatTensor(testY).to(device)

((26280, 6), (672, 6))

In [ ]:
# loss & optimizer setting
net = Net(data_dim, hidden_dim, output_dim, 4).to(device)

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# start training
net.train()
best_loss = int(1e9)

for i in range(iterations):
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_model = net

best_model.eval()
dangjin_warehouse_pred = []
for i in range(672):
    x_input  = df[-672-24+i:-672+i].reshape(1,24,6)

    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = best_model(x_input).cpu().detach().numpy()

    dangjin_warehouse_pred.append(predict[0][0])
    new_input = predict.reshape((1))
    df[-672-24+i:-672+i][-1][-1] = new_input[0]
  

Epoch 0, Loss 29104.04688
Epoch 100, Loss 21044.23633
Epoch 200, Loss 21043.84180
Epoch 300, Loss 21043.81641
Epoch 400, Loss 2698.15503
Epoch 500, Loss 1220.55444
Epoch 600, Loss 1014.64471
Epoch 700, Loss 891.23059
Epoch 800, Loss 840.61072
Epoch 900, Loss 792.05908
Epoch 1000, Loss 714.65308
Epoch 1100, Loss 624.62286
Epoch 1200, Loss 561.84656
Epoch 1300, Loss 511.23148
Epoch 1400, Loss 481.55777
Epoch 1500, Loss 508.91702
Epoch 1600, Loss 504.68076
Epoch 1700, Loss 428.37595
Epoch 1800, Loss 352.46747
Epoch 1900, Loss 286.81018


In [ ]:
df = train_datast(energy, dangjin_interpolated, target='dangjin')

train_size = int(len(df) * 1)
train_set = df[:train_size]
test_set = df[train_size-672:]
train_set.shape, test_set.shape


# train-test dataset to input

trainX, trainY = make_sequence(train_set, seq_length)
testX, testY = make_sequence(test_set, seq_length)

# convert to tensor
trainX_tensor = torch.FloatTensor(trainX).to(device)
trainY_tensor = torch.FloatTensor(trainY).to(device)

testX_tensor = torch.FloatTensor(testX).to(device)
testY_tensor = torch.FloatTensor(testY).to(device)

In [ ]:
# loss & optimizer setting
net = Net(data_dim, hidden_dim, output_dim, 4).to(device)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# start training
net.train()
best_loss = int(1e9)

for i in range(iterations):
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_model = net

best_model.eval()
dangjin_pred = []
for i in range(672):
    x_input  = df[-672-24+i:-672+i].reshape(1,24,6)

    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = best_model(x_input).cpu().detach().numpy()

    dangjin_pred.append(predict[0][0])
    new_input = predict.reshape((1))
    df[-672-24+i:-672+i][-1][-1] = new_input[0]
  

## 울산


In [20]:
df = train_datast(energy, ulsan_interpolated, target='ulsan')

train_size = int(len(df) * 1)
train_set = df[:train_size]
test_set = df[train_size-672:]
train_set.shape, test_set.shape


# train-test dataset to input

trainX, trainY = make_sequence(train_set, seq_length)
testX, testY = make_sequence(test_set, seq_length)

# convert to tensor
trainX_tensor = torch.FloatTensor(trainX).to(device)
trainY_tensor = torch.FloatTensor(trainY).to(device)

testX_tensor = torch.FloatTensor(testX).to(device)
testY_tensor = torch.FloatTensor(testY).to(device)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
# loss & optimizer setting
net = Net(data_dim, hidden_dim, output_dim, 4).to(device)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# start training
net.train()
best_loss = int(1e9)

for i in range(iterations):
    outputs = net(trainX_tensor)
    loss = criterion(outputs, trainY_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_model = net

best_model.eval()
ulsan_pred = []
for i in range(672):
    x_input  = df[-672-24+i:-672+i].reshape(1,24,6)

    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = best_model(x_input).cpu().detach().numpy()

    ulsan_pred.append(predict[0][0])
    new_input = predict.reshape((1))
    df[-672-24+i:-672+i][-1][-1] = new_input[0]
  

Epoch 0, Loss 13939.99902
